f'/Volumes/resabi202400022-mri-harmonization-data/data/roshan/combat/harmonised_images/{source}/'

# Combat Implicit harmonisation

# Making the 1000 matrices

## Uses site, age, sex as covars 
other thing only used site

In [1]:
import numpy as np
import pandas as pd
from scipy.io import loadmat, savemat
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, classification_report,recall_score, precision_score, 
                             roc_auc_score, roc_curve, confusion_matrix, balanced_accuracy_score)
import os
import glob
from neuroCombat import neuroCombat


In [2]:
# control_path = '/Volumes/resabi202400022-mri-harmonization-data/data/fmri/abide/'
control_path = '/Users/roshan/Desktop/fMRI/abide/'
# Use glob to search for the specific .mat file recursively
specific_files = glob.glob(os.path.join(control_path, '**', '*_AAL116_correlation_matrix.mat'), recursive=True)

# # Print the list of found file
patient_files = [f for f in specific_files if 'patient' in os.path.dirname(f)]
control_files = [f for f in specific_files if 'control' in os.path.dirname(f)]


In [3]:
# meta_data = pd.read_csv('ABIDE.csv')
meta_data = pd.read_csv('Sheet 1-ABIDE_metadata.csv')
meta_data.head()

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Site
0,I327154,50002,Autism,M,17,1,fMRI,Resting State fMRI,PITT
1,I328631,50002,Autism,M,17,1,MRI,MP-RAGE,PITT
2,I328256,50003,Autism,M,24,1,fMRI,Resting State fMRI,PITT
3,I328410,50003,Autism,M,24,1,MRI,MP-RAGE,PITT
4,I328736,50004,Autism,M,19,1,MRI,MP-RAGE,PITT


In [4]:
abide_df = meta_data[['Subject', 'Site', 'Sex', 'Age']]



In [5]:
def extract_subject_id(file_path):
    # Assumes the subject ID is always after 'sub-control' or 'sub-patient' and is a numeric value
    base_name = os.path.basename(file_path)  # Get the file name (e.g., 'sub-control50197_AAL116_correlation_matrix.mat')
    subject_id = base_name.split('_')[0].replace('sub-control', '').replace('sub-patient', '')  # Extract the subject ID
    return int(subject_id)


In [6]:
# abide_df.head()
all_files = control_files+patient_files

In [7]:
def load_matrix(file_path):
    mat_data = loadmat(file_path)
    return mat_data['data']  # Adjust 'data' to match the key in your .mat files


In [8]:
abide_df.loc[abide_df['Subject'] == 50002]

,Subject,Site,Sex,Age
0,50002,PITT,M,17
1,50002,PITT,M,17


In [9]:
data_matrices = []
labels = []
subject_ids = []
site_info = []
sex_info = []
age_info = []
# Process control files (label = 0)
for file_path in control_files:
    subject_id = extract_subject_id(file_path)
    
    # Look up site information from `abide_df` using subject ID
    row = abide_df.loc[abide_df['Subject'] == subject_id]
    site = row['Site'].values[0]
    sex = row['Sex'].values[0]
    age = row['Age'].values[0]
    
    
    # Load the matrix and flatten it
    matrix = load_matrix(file_path)
    data_matrices.append(matrix.flatten())
    labels.append(0)  # Control label
    subject_ids.append(subject_id)
    site_info.append(site)
    sex_info.append(sex)
    age_info.append(age)

# Process autism files (label = 1)
for file_path in patient_files:
    subject_id = extract_subject_id(file_path)
    
    # Look up site information from `abide_df` using subject ID
    row = abide_df.loc[abide_df['Subject'] == subject_id]
    site = row['Site'].values[0]
    sex = row['Sex'].values[0]
    age = row['Age'].values[0]
    # Load the matrix and flatten it
    matrix = load_matrix(file_path)
    data_matrices.append(matrix.flatten())
    labels.append(1)  # Autism label
    subject_ids.append(subject_id)
    site_info.append(site)
    sex_info.append(sex)
    age_info.append(age)

In [10]:
# Convert lists to numpy arrays
data_matrices = np.array(data_matrices)
labels = np.array(labels)
site_info = np.array(site_info)
sex_info = np.array(sex_info)
age_info = np.array(age_info)

# Prepare the site DataFrame for ComBat harmonization
covars = pd.DataFrame({
    'site': site_info,
    'sex': sex_info,  # Include sex as a covariate
    'age': age_info   # Include age as a covariate
})

In [11]:
# Perform ComBat harmonization
combat_harmonized_data = neuroCombat(
    dat=data_matrices.T,  # Transpose so that features are rows
    covars=covars,
    batch_col='site',
    categorical_cols=['sex'],  # Sex is categorical
    continuous_cols=['age']    # Age is continuous
)['data']

[neuroCombat] Creating design matrix
[neuroCombat] Standardizing data across features
[neuroCombat] Fitting L/S model and finding priors
[neuroCombat] Finding parametric adjustments
[neuroCombat] Final adjustment of data


In [12]:
# Transpose back so subjects are rows
combat_harmonized_data = combat_harmonized_data.T

# Reshape the harmonized data back to 116x116 matrices
harmonized_matrices = combat_harmonized_data.reshape(-1, 116, 116)


In [13]:
 #Save the harmonized matrices to disk
output_dir = '/Users/roshan/Desktop/fMRI/output/combat_new/'
os.makedirs(output_dir, exist_ok=True)
file_paths = []

for i, subject_id in enumerate(subject_ids):
    # Determine the prefix based on the autism label (0 for control, 1 for autism)
    if labels[i] == 0:
        prefix = 'sub-control'
    else:
        prefix = 'sub-patient'
    
    # Construct the file name
    file_name = f'{prefix}{subject_id}_harmonized.mat'
    file_path = os.path.join(output_dir, file_name)
    
    # Save as .mat file with the correct file name
    savemat(file_path, {'data': harmonized_matrices[i]})
    file_paths.append(file_path)

# Create a CSV file to store file paths, labels, subject IDs, and site info
output_csv = '/Users/roshan/Desktop/fMRI/output/combat_new/harmonized_data_info.csv'
csv_df = pd.DataFrame({
    'file_path': file_paths,
    'subject_id': subject_ids,
    'autism_label': labels,
    'site': site_info
})

csv_df.to_csv(output_csv, index=False)
print(f"Harmonized data info saved to: {output_csv}")

Harmonized data info saved to: /Users/roshan/Desktop/fMRI/output/combat_new/harmonized_data_info.csv
